In [ ]:
import numpy as np, sys,os
import pandas as pd
import matplotlib.pyplot as plt
import sys
from tqdm import tqdm
sys.path.insert(0, "..")
import random
import src.utils.dtw as dtw
import src.models.augmentations as augmentations
import src.visualization.visualize as visualize
import src.utils.data_extraction as extractions
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LeakyReLU, BatchNormalization, Reshape, Flatten
from tensorflow.keras.optimizers import Adam


In [ ]:
# Longitud de la serie de tiempo a generar
TIME_STEPS = 500

# Dimensión de entrada para el Generador
NOISE_DIM = 20

# Número de unidades en las capas ocultas
HIDDEN_UNITS = 128

# Tasa de aprendizaje para el optimizador
LEARNING_RATE = 0.0002

# Número de iteraciones de entrenamiento
EPOCHS = 10000

# Tamaño del lote para el entrenamiento del GAN
BATCH_SIZE = 64

# Función para generar ruido (entrada del Generador)
def generate_noise(batch_size, dim):
    return np.random.normal(0, 1, size=(batch_size, dim))

# Función para generar datos de series de tiempo reales (no usada en este caso)
def generate_real_data(batch_size):
    return np.random.uniform(-1, 1, size=(batch_size, TIME_STEPS, 1))

# Función para mostrar algunas series de tiempo generadas
def plot_generated_time_series(samples):
    plt.figure(figsize=(10, 6))
    for i in range(samples.shape[0]):
        plt.plot(samples[i], linewidth=1)
    plt.title('Series de Tiempo Generadas por el GAN')
    plt.xlabel('Tiempo')
    plt.ylabel('Valor')
    plt.show()


In [ ]:
# Construir el Generador
generator = Sequential([
    Dense(units=HIDDEN_UNITS, input_dim=NOISE_DIM),
    LeakyReLU(alpha=0.2),
    BatchNormalization(),
    Dense(units=TIME_STEPS * 1, activation='tanh'),
    Reshape((TIME_STEPS, 1))
])

# Construir el Discriminador
discriminator = Sequential([
    Flatten(input_shape=(TIME_STEPS, 1)),
    Dense(units=HIDDEN_UNITS),
    LeakyReLU(alpha=0.2),
    Dense(units=1, activation='sigmoid')
])


In [ ]:
# Compilar el Generador
generator.compile(loss='binary_crossentropy', optimizer=Adam(lr=LEARNING_RATE))

# Congelar el Discriminador para el entrenamiento del GAN
discriminator.trainable = False

# Compilar el GAN (Generador + Discriminador)
gan_input = tf.keras.Input(shape=(NOISE_DIM,))
generated_series = generator(gan_input)
gan_output = discriminator(generated_series)
gan = Model(gan_input, gan_output)
gan.compile(loss='binary_crossentropy', optimizer=Adam(lr=LEARNING_RATE))


In [ ]:
# Función para entrenar el GAN
def train_gan(X_train, epochs, batch_size, sample_interval=100):
    for epoch in range(epochs):
        # ---------------------
        #  Entrenar Discriminador
        # ---------------------

        # Seleccionar un lote aleatorio de datos reales
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        real_series = X_train[idx]

        # Generar ruido como entrada del Generador
        noise = generate_noise(batch_size, NOISE_DIM)

        # Generar un lote de series de tiempo falsas con el Generador
        generated_series = generator.predict(noise)

        # Entrenar el Discriminador
        d_loss_real = discriminator.train_on_batch(real_series, np.ones((batch_size, 1)))
        d_loss_fake = discriminator.train_on_batch(generated_series, np.zeros((batch_size, 1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # ---------------------
        #  Entrenar Generador
        # ---------------------

        # Generar ruido como entrada del GAN
        noise = generate_noise(batch_size, NOISE_DIM)

        # Entrenar el Generador (el Discriminador se mantiene fijo)
        g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))

        # Mostrar el progreso del entrenamiento
        if epoch % sample_interval == 0:
            print(f'Epoch {epoch}, Discriminador Loss: {d_loss}, Generador Loss: {g_loss}')
            # Generar y mostrar ejemplos de series de tiempo generadas
            samples = generator.predict(generate_noise(5, NOISE_DIM))
            plot_generated_time_series(samples)


In [ ]:
timeSeries = pd.read_json('../data/processed/realData.json', orient='records', lines=True)
# Filtrar las series de tiempo por clase
healthy_cop_x = timeSeries[timeSeries['class'] == 'Healthy']['cop_x']
neuropathic_cop_x = timeSeries[timeSeries['class'] == 'Neuropathic']['cop_x']
diabetic_cop_x = timeSeries[timeSeries['class'] == 'Diabetic']['cop_x']

healthy_cop_y = timeSeries[timeSeries['class'] == 'Healthy']['cop_y']
neuropathic_cop_y = timeSeries[timeSeries['class'] == 'Neuropathic']['cop_y']
diabetic_cop_y = timeSeries[timeSeries['class'] == 'Diabetic']['cop_y']

In [ ]:
# Generar datos de ejemplo para entrenar el GAN
X_train = healthy_cop_x  # En este caso, generamos datos aleatorios uniformemente distribuidos

# Normalizar los datos para que estén entre -1 y 1
X_train = (X_train - 0.5) / 0.5

# Entrenar el GAN
train_gan(X_train, EPOCHS, BATCH_SIZE)
